# * Import : "Metric Master List"

## Parameter

In [1]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import FN_CFW as fn

# Input parameter
src_dir = 'data'
src_file = 'Metric Master List'
src_sheet = 'DIM_METRIC_DATA'
curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Rawdata

In [2]:
data_src = f'{src_dir}/{src_file}.xlsx'
src_df = pd.read_excel(data_src, sheet_name=src_sheet, index_col=None) 
# , skiprows=2, usecols='N:AD'
src_df.head(3)

,METRIC_CD,METRIC_NAME,METRIC_ID,COMP_CD,CENTER,AGG_TYPE,RR_IND,CONS_IND,METRIC_GRP,METRIC_SUB_GRP,...,Network,Sales,Experience,Mobile,Prepaid,Postpaid,TOL,TVS,TDG,B2B
0,B0R000100,Total Revenue,B0R000100,ALL,Revenue,S,1.0,N,Revenue,Revenue,...,No,Yes,No,No,No,No,No,No,No,No
1,B0R00010001,Total Inflow M1,B0R00010001,ALL,Sales,S,1.0,S,Sales,Sales,...,No,Yes,No,No,No,No,No,No,No,No
2,B0R00010001AA,Total Inflow M1 : Account Executive,B0R00010001AA,ALL,Sales,S,0.0,N,Sales,Sales,...,No,Yes,No,No,No,No,No,No,No,No


In [17]:
cols = ['METRIC_CD', 'METRIC_NAME', 'METRIC_ID', 'COMP_CD', 'CENTER', 'AGG_TYPE', 'RR_IND', 'CONS_IND', 'METRIC_GRP', 'METRIC_SUB_GRP', 'PRODUCT_GRP', 'GRY_IND', 'UOM', 'DEF', 'CAL', 
        'CORPORATE', 'MICRO_ORG', 'METRIC_TYPE', 'RADAR', 'KPI_TYPE', 'LIST', 'REMARK', 
        'COR_EQUAL_NAT', 'NATIONWIDE', 'REGION', 'PROVINCE', 'CLUSTER', 'DISTRICT', 'SUBDISTRICT', 'Data Source']

tmp_df = src_df[cols]
# tmp_df = src_df
tmp_df['MY_GROUP'] = tmp_df.apply(lambda x: fn.my_metric_group(str(x['METRIC_GRP']), str(x['METRIC_CD']), str(x['METRIC_NAME'])), axis=1)
# tmp_df['MY_GROUP'].unique()

# Filter
tmp_df = tmp_df.loc[tmp_df['CENTER'].isin(['Revenue', 'Sales', 'Profitability'])]
tmp_df = tmp_df.loc[~tmp_df['METRIC_CD'].str.contains('[0-9]C$|[0-9]H$|[0-9]MCOM$|[0-9]CUS$')]
tmp_df = tmp_df.loc[~tmp_df['METRIC_NAME'].str.contains('\(Customer Location\)')]
tmp_df = tmp_df.loc[~tmp_df['METRIC_CD'].str.contains('[0-9]A[A-K]$')]
tmp_df = tmp_df.loc[tmp_df['METRIC_CD'].str.contains('CORP$|GEO$')]
# tmp_df = tmp_df.loc[~tmp_df['METRIC_NAME'].str.contains('\(Geo\)')]
# tmp_df = tmp_df.loc[tmp_df['PRODUCT_GRP'].isin(['All Services', 'TOL'])]
# tmp_df = tmp_df.loc[tmp_df['PRODUCT_GRP']=='All Services']
# tmp_df = tmp_df.loc[tmp_df['PRODUCT_GRP']=='Prepaid']
# tmp_df = tmp_df.loc[tmp_df['METRIC_GRP']=='Revenue']
tmp_df = tmp_df.loc[tmp_df['METRIC_GRP'].isin(['Revenue', 'Sales', 'Subs'])]
# tmp_df = tmp_df.loc[tmp_df['MY_GROUP'].isin(['Revenue', 'Inflow M1', 'Gross Adds', 'Inflow M2', '%Conversion', 'GA ARPU/RC', 'Active Subs', 'ARPU', 'New/Existing'])]
# tmp_df = tmp_df.loc[tmp_df['MY_GROUP']=='Revenue']

# tmp_df = tmp_df.loc[tmp_df['METRIC_GRP']=='Revenue']
# tmp_df = tmp_df.loc[tmp_df['METRIC_GRP']=='Sales']
# tmp_df = tmp_df.loc[tmp_df['METRIC_GRP']=='Subs']
# tmp_df = tmp_df.loc[tmp_df['METRIC_NAME'].str.contains('TDG')]

# src_df = src_df.replace(np.nan, None)
tmp_df = tmp_df.reset_index(drop=True)

tmp_df#.tail()

C:\Users\Narut4\AppData\Local\Temp\ipykernel_12732\2707973792.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['MY_GROUP'] = tmp_df.apply(lambda x: fn.my_metric_group(str(x['METRIC_GRP']), str(x['METRIC_CD']), str(x['METRIC_NAME'])), axis=1)


,METRIC_CD,METRIC_NAME,METRIC_ID,COMP_CD,CENTER,AGG_TYPE,RR_IND,CONS_IND,METRIC_GRP,METRIC_SUB_GRP,...,REMARK,COR_EQUAL_NAT,NATIONWIDE,REGION,PROVINCE,CLUSTER,DISTRICT,SUBDISTRICT,Data Source,MY_GROUP
0,B0R00010001GEO,Total Inflow M1 (Geo),B0R00010001GEO,ALL,Sales,S,1.0,N,Sales,Sales,...,NaN,No,Yes,Yes,Yes,Yes,No,No,DWH (Calculation),Inflow M1
1,B0R000100GEO,Total Revenue (Geo),B0R000100GEO,ALL,Revenue,S,1.0,N,Revenue,Revenue,...,Total Revenue (Geo) is a daily actual revenue ...,No,Yes,Yes,Yes,No,No,No,DWH (Calculation),Revenue
2,B1R000100CORP,Prepaid Revenue,B1R000100CORP,ALL,Revenue,S,1.0,N,Revenue,Revenue,...,NaN,No,Yes,No,No,No,No,No,DWH (Calculation),Revenue
3,B1R000101CORP,Prepaid New Revenue,B1R000101CORP,ALL,Revenue,S,1.0,N,Revenue,Revenue,...,NaN,No,Yes,No,No,No,No,No,NaN,New/Existing
4,B1R000102CORP,Prepaid Existing Revenue,B1R000102CORP,ALL,Revenue,S,1.0,N,Revenue,Revenue,...,NaN,No,Yes,No,No,No,No,No,NaN,New/Existing
5,B1R000900GEO,Prepaid Inflow M1 (Geo),B1R000900GEO,ALL,Sales,S,1.0,N,Sales,Sales,...,NaN,No,Yes,Yes,Yes,Yes,No,No,DWH (Calculation),Inflow M1
6,B1R001000GEO,Prepaid Inflow M2 (Geo),B1R001000GEO,ALL,Sales,S,1.0,N,Sales,Sales,...,NaN,No,Yes,Yes,Yes,Yes,No,No,DWH (Calculation),Inflow M2
7,B1R001100GEO,Prepaid Conversion M2 to M1 (Geo),B1R001100GEO,ALL,Sales,N,0.0,N,Sales,Sales,...,NaN,No,Yes,Yes,Yes,No,No,No,NaN,%Conversion
8,B1R001200GEO,Prepaid GA ARPU (Geo),B1R001200GEO,ALL,Sales,N,1.0,N,Sales,Sales,...,NaN,No,Yes,Yes,Yes,No,No,No,DWH (Calculation),GA ARPU/RC
9,B1S000101GEO,Prepaid Gross Adds (Geo),B1S000101GEO,ALL,Sales,S,1.0,N,Sales,Sales,...,NaN,No,Yes,Yes,Yes,Yes,No,No,DWH (Calculation),Gross Adds


In [9]:
# tmp_df = src_df[cols]
# tmp_df = tmp_df.loc[tmp_df['METRIC_ID']=='B0R000100']
# tmp_df = tmp_df.reset_index(drop=True)
# tmp_df[['METRIC_CD', 'METRIC_NAME', 'METRIC_ID', 'COMP_CD', 'CENTER', 'AGG_TYPE', 'RR_IND', 'CONS_IND', 'METRIC_GRP', 'METRIC_SUB_GRP', 'PRODUCT_GRP', 'GRY_IND']]

In [10]:
# tmp_df = src_df[cols]
# tmp_df = tmp_df.loc[tmp_df['METRIC_ID']=='B0R000100GEO']
# tmp_df = tmp_df.reset_index(drop=True)
# tmp_df[['METRIC_CD', 'METRIC_NAME', 'METRIC_ID', 'COMP_CD', 'CENTER', 'AGG_TYPE', 'RR_IND', 'CONS_IND', 'METRIC_GRP', 'METRIC_SUB_GRP', 'PRODUCT_GRP', 'GRY_IND']]

## Generate Output file

In [12]:
# to Excel file

op_dir = 'temp'
op_file = 'TMP_Metric Master List.xlsx'

# tmp_df.to_excel(f'{op_dir}/{op_file}', sheet_name='Data', index=False)
# print(f'\n  -> Generate "{op_file}" successfully')


  -> Generate "TMP_Metric Master List.xlsx" successfully
